<a href="https://colab.research.google.com/github/lucastephan15/kk/blob/main/Python_Real_Estate_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 32.0 MB/s eta 0:00:00


In [ ]:
pip install dash-bootstrap-components


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
pip install pandas

In [ ]:
pip install plotly

In [ ]:
pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00


In [14]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objs as go

# Your existing Python code components
criteria_weights = {
    'Location': {
        'weights': {
            'Accessibility': 0.1,
            'Amenities': 0.1,
            'Safety': 0.1,
            'Growth Potential': 0.1
        },
        'total_weight': 0.4  # Total weight for Location category
    },
    'Development Concept and Market Fit': {
        'weights': {
            'Innovation': 0.0625,
            'Design Quality': 0.0625,
            'Target Market Alignment': 0.0625,
            'Unique Selling Proposition': 0.0625
        },
        'total_weight': 0.25  # Total weight for this category
    },
    'Financing and Investment Structure': {
        'weights': {
            'ROI Potential': 0.0375,
            'Funding Stability': 0.0375,
            'Financial Incentives': 0.0375,
            'Risk Assessment': 0.0375
        },
        'total_weight': 0.15
    },
    'Density per Home': {
        'weights': {
            'Space Utilization': 0.025,
            'Community Density': 0.025,
            'Privacy Considerations': 0.025,
            'Infrastructure Load': 0.025
        },
        'total_weight': 0.1
    },
    'Matching Criteria': {
        'weights': {
            'Demographic Fit': 0.025,
            'Demand Forecast': 0.025,
            'Competitor Analysis': 0.025,
            'Market Saturation': 0.025
        },
        'total_weight': 0.1
    },
    'Supply and Demand': {
        'weights': {
            'Current Market Conditions': 0.0125,
            'Projected Supply Changes': 0.0125,
            'Demand Growth Rate': 0.0125,
            'Economic Indicators': 0.0125
        },
        'total_weight': 0.05
    },
    'Benchmarking of Sales Compared to Competitors': {
        'weights': {
            'Price Competitiveness': 0.0125,
            'Sales Velocity': 0.0125,
            'Market Share': 0.0125,
            'Customer Satisfaction': 0.0125
        },
        'total_weight': 0.05
    },
    'Rating of the Company Behind the Project': {
        'weights': {
            'Reputation': 0.0125,
            'Financial Health': 0.0125,
            'Operational Excellence': 0.0125,
            'Social Responsibility': 0.0125
        },
        'total_weight': 0.05
    },
    # Additional criteria can be added here in a similar structure
}

# Placeholder for inputting scores, replace None with your scores
project_scores = {
    'Location': {
        'Accessibility': None,
        'Amenities': None,
        'Safety': None,
        'Growth Potential': None
    },
    'Development Concept and Market Fit': {
        'Innovation': None,
        'Design Quality': None,
        'Target Market Alignment': None,
        'Unique Selling Proposition': None
    },
    'Financing and Investment Structure': {
        'ROI Potential': None,
        'Funding Stability': None,
        'Financial Incentives': None,
        'Risk Assessment': None
    },
    'Density per Home': {
        'Space Utilization': None,
        'Community Density': None,
        'Privacy Considerations': None,
        'Infrastructure Load': None
    },
    'Matching Criteria': {
        'Demographic Fit': None,
        'Demand Forecast': None,
        'Competitor Analysis': None,
        'Market Saturation': None
    },
    'Supply and Demand': {
        'Current Market Conditions': None,
        'Projected Supply Changes': None,
        'Demand Growth Rate': None,
        'Economic Indicators': None
    },
    'Benchmarking of Sales Compared to Competitors': {
        'Price Competitiveness': None,
        'Sales Velocity': None,
        'Market Share': None,
        'Customer Satisfaction': None
    },
    'Rating of the Company Behind the Project': {
        'Reputation': None,
        'Financial Health': None,
        'Operational Excellence': None,
        'Social Responsibility': None
    },

}

def calculate_weighted_scores(criteria_weights, project_scores):
    category_scores = {}
    for category, details in criteria_weights.items():
        subcriteria = details['weights']
        category_score = 0
        for sub in subcriteria:
            if project_scores[category][sub] is not None:
                category_score += project_scores[category][sub] * subcriteria[sub]
        if sum(subcriteria.values()) > 0:
            category_score = category_score * details['total_weight'] / sum(subcriteria.values())
        category_scores[category] = category_score
    return category_scores


def determine_final_score_and_tier(category_scores, tier_boundaries):
    final_score = sum(category_scores.values())
    tier = 'Tier ' + str(len(tier_boundaries) + 1)  # Default to the last tier
    for i, boundary in enumerate(tier_boundaries):
        if final_score <= boundary:
            tier = 'Tier ' + str(i + 1)
            break
    return final_score, tier

# Initialize Dash app with Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define a function to generate a card for each category
def generate_category_card(category, details):
    return dbc.Card(
        dbc.CardBody([
            html.H5(category, className="card-title"),
            dbc.Row([
                dbc.Col(dcc.Input(id=f"{category}-{subcriterion}", type="number", placeholder=f" {subcriterion}"), width=4)
                for subcriterion in details['weights']
            ], className="mb-3")
        ])
    )

# Define the layout of the app with Bootstrap components
app.layout = dbc.Container([
    html.H1("Project Evaluation Dashboard", className="mb-5 mt-5"),
    dbc.Row([
        dbc.Col(generate_category_card(category, details), md=4)
        for category, details in criteria_weights.items()
    ], className="mb-3"),
    dbc.Button('Calculate Scores', id='calculate-scores', color="primary", className="me-md-2", n_clicks=0),
    html.Div(id='final-score-and-tier', className="mt-3"),
    dbc.Row([
        dbc.Col(html.Div(id=f"{category}-score"), width={"size": 6, "offset": 3})
        for category in criteria_weights
    ], className="mb-3")
], fluid=True)

# Callbacks remain largely the same with minor adjustments for Bootstrap compatibility...

@app.callback(
    [Output('final-score-and-tier', 'children')] +
    [Output(f"{category}-score", 'children') for category in criteria_weights],
    [Input('calculate-scores', 'n_clicks')],
    [Input(f"{category}-{subcriterion}", 'value') for category in criteria_weights for subcriterion in criteria_weights[category]['weights']]
)
def update_scores(n_clicks, *input_values):
    if n_clicks > 0:
        # Update project_scores based on input values
        input_iter = iter(input_values)
        for category in criteria_weights:
            for subcriterion in criteria_weights[category]['weights']:
                project_scores[category][subcriterion] = next(input_iter)

        # Calculate category scores and final score and tier
        category_scores = calculate_weighted_scores(criteria_weights, project_scores)
        final_score, tier = determine_final_score_and_tier(category_scores, tier_boundaries=[1, 2, 3, 4])

        # Prepare output for display
        final_score_display = f'Final Score: {final_score:.2f}, Assigned Tier: {tier}'
        category_scores_display = [f'{category} Score: {category_scores[category]:.2f}' for category in category_scores]

        return (final_score_display, *category_scores_display)

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>